## **Import Required Libraries**

In [ ]:
# Import pandas
import pandas as pd

# Import libraries for web scraping
import requests
from bs4 import BeautifulSoup

# Import wikipedia
import wikipedia

# Import matplotlib for plotting
import matplotlib.pyplot as plt

# Import json to load config file
import json

## **Creates UK Election Data**

### **Import Raw Data**

In [ ]:
# Load the file path from the config.json file
with open('config.json', 'r') as file:
    config = json.load(file)

# Access the input directory path
inp_dir = config['input_data_dir']

# Import recent election results raw data
uk_re_raw = pd.read_csv(f"{inp_dir}UK_Election_Results.csv")

### **Clean Data**

In [ ]:
# Remove parties with 0 vote or 0 change
uk_re = uk_re_raw[(uk_re_raw["Vote Share"] != 0) | (uk_re_raw["Change Since 2019"] != 0)].copy()

### **Add Alignment of Each Party**

In [ ]:
# Create mapping from party to political alignment (using Wikipedia)
party_alignment = {
    "Labour": "Centre-left",
    "Conservative": "Centre-right to right-wing",
    "Reform UK": "Right-wing",
    "Liberal Democrat": "Centre to centre-left",
    "Green": "Left-wing",
    "Scottish National Party": "Centre-left",
    "Plaid Cymru": "Centre-left to left-wing",
    "Sinn Féin": "Centre-left to left-wing",
    "Workers Party of Britain": "Left-wing to far-left",
    "Democratic Unionist Party": "Right-wing"
}

# Create numerical mapping of Alignment
#   ranges from -6 (Far-Left) to 6 (Far-Right)
alignment_mapping = {
    "Centre-left": -2,
    "Centre-right": 2,
    "Centre": 0,
    "Right-wing": 4,
    "Left-wing": -4,
    "Centre to centre-left": -1,
    "Centre to centre-right": 1,
    "Centre-right to right-wing": 3,
    "Center-left": -2,
    "Big tent": 0,
    "Centre-left to left-wing": -3,
    "Far-left": -6,
    "Centre-rightFactions:Right-wing": 3,
    "CentreFactions:Left-wing and right-wing": 0,
    "\nMajority:\nRight-wing": 4,
    "Right-wing to far-right": 5,
    "Center-right": 2,
    "Center-right to right-wing": 3,
    "Far-right": 6,
    "Left-wing to far-left": -5,
    None: "No Data",        # PLACEHOLDER: CHANGE!
}

# Map Party to alignment
uk_re["Alignment"] = uk_re["Party"].map(party_alignment)

# Map alignment to numerical alignment
uk_re["Alignment Score"] = uk_re["Alignment"].map(alignment_mapping)

# Check that all valid parties have been mapped
print(f"Number of unmapped alignments: {len(uk_re[(~uk_re['Alignment'].isna()) & (uk_re['Alignment Score'].isna())])}")

# View data
uk_re

### **Split Data into Two**

In [ ]:
# Create 2019 vote count col
uk_re["2019 Vote Share"] = uk_re["Vote Share"] - uk_re["Change Since 2019"]

# Create 2024 data
uk_re_2024 = uk_re[["Alignment Score", "Vote Share"]].copy()
uk_re_2024.rename(columns = {"Vote Share": "In-Scope Vote Prop"}, inplace=True)

# Create 2019 data
uk_re_2019 = uk_re[["Alignment Score", "2019 Vote Share"]].copy()
uk_re_2019.rename(columns = {"2019 Vote Share": "In-Scope Vote Prop"}, inplace=True)

## **Data Visualisation**

### **Give Alignment Score**

In [ ]:
# Give alignment score for Uk 2024 election
align_score_2024 = uk_re_2024["Alignment Score"].dot(uk_re_2024["In-Scope Vote Prop"])
print(f"Alignment score of the 2024 Uk Election: {align_score_2024}\n")

# Give alignment score for UK 2019 election
align_score_2019 = uk_re_2019["Alignment Score"].dot(uk_re_2019["In-Scope Vote Prop"])
print(f"Alignment score of the 2019 UK Election: {align_score_2019}")

## **Export Data**

In [ ]:
# Access the input directory path
out_dir = config['inter_outputs_dir']

uk_re_2019.to_csv(f"{out_dir}UK_Prev.csv", index=False)
uk_re_2024.to_csv(f"{out_dir}UK_New.csv", index=False)